# IE2 Big Data Project (weilemar & vongrdir)

Im folgenden Jupyter Notebook wird der Code vorgestellt, welcher verwendet wurde um das [Homocide Report](https://www.kaggle.com/murderaccountability/homicide-reports) Dataset zu analysieren. Der Auftraggeber auf Kaggle ist die Organisation Murder Accountability Project (http://www.murderdata.org/).
In diesem Datensatz befinden sich 23 Attribute für 638‘454 Morde in Amerika von 1980-2014. Ziel des Kaggle Aufrages ist es, ein Muster in den Daten für "Serial-Killer" zu entdecken. Mit diesen Daten könnte man auch sehr viele, spannende Diagramme erstellen. Jedoch liegt unser Fokus in dieser Arbeit auf dem clustern von Daten.
Konkret möchten wir mit 2 verschiednen Daten Cluster mit k-means erstellen. Einerseits verwenden wir die Rohdaten in One-hot-encoding Form. Andererseits verwenden wir Embeddings welche mit Hilfe unserers zuvor trainierten Autoencoder generiert werden. Wir möchten die resultierenden Cluster vergleichen und auf beiden Resultaten anschliessend noch mit Hilfe einer PCA herausfinden, welche Attribute ausschlaggebend waren.

Als erster Schritt muss das rohe Dataset über die folgende URL heruntergeladen werden:

In [4]:
%%bash
cd data
wget -O homocide-reports.zip https://github.com/vongruenigen/IE2-Project/raw/master/data/homicide-reports.zip
unzip -f homocide-reports.zip

Archive:  homocide-reports.zip


--2017-05-26 16:42:02--  https://github.com/vongruenigen/IE2-Project/raw/master/data/homicide-reports.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vongruenigen/IE2-Project/master/data/homicide-reports.zip [following]
--2017-05-26 16:42:03--  https://raw.githubusercontent.com/vongruenigen/IE2-Project/master/data/homicide-reports.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11010328 (10M) [application/octet-stream]
Saving to: ‘homocide-reports.zip’

     0K .......... .......... .......... .......... ..........  0%  811K 13s
    50K .......... .......... .......... .......... ..........  0% 1.88M 9s
   1

Nachdem das Dataset heruntergeladen und entpackt ist muss daraus mittels des preprocessing Skripts (für Original siehe scripts/preprocess_data.py) das eigentliche Dataset mit One-Hot Encoded Vektoren verwendet werden. Zuerst aber einige Hilfsfunktionen, Import von Bibliotheken und Definitionen von Konstanten.

In [ ]:
import os
import re
import sys
import time
import tensorflow as tf
import numpy as np

from collections import defaultdict
from operator import itemgetter
from os import path

# Constants for file names
PROJECT_HOME = '/media/dvg/Volume/Dropbox/ZHAW/IE2/Project'
RAW_DATA_PATH = 'data/database.csv'
PREPROCESSED_DATA_PATH = 'data/samples.csv'

# Hyperparameters for autoencoder
TRAINING_EPOCHS = 1000
BATCH_SIZE = 128
DISPLAY_EPOCH = 1
DISPLAY_BATCH = 1000
HIDDEN_SIZE = 256
RESULTS_DIR = path.abspath(path.join(PROJECT_HOME, 'results'))

# Columns we are going to ignore in the dataset because they're redundant or non-informative
STRIP_COLS = ('Record ID', 'Agency Name', 'Agency Code', 'Year', 'Month', 'Record Source')

def camel_to_sneak(name):
    '''Convert a string from camel-case to sneak-case.'''
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def error(msg):
    '''Logs an error message and terminates the process.'''
    log(msg, level='error')

def log(msg, level='info'):
    '''Logs the given message with the given level.'''
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    print('[%s][%s] %s' % (level.upper(), ts, msg))
    if level == 'error': sys.exit(2)
        

def get_next_batch(train_data_f, batch_size):
    '''Returns the next batch of vectors from the given CSV file.'''
    new_batch = []

    for _ in range(batch_size):
        new_line = train_data_f.readline().strip('\n')
        new_batch.append(np.array((map(float, new_line.split(';')))))

    return new_batch

def get_input_size_and_length(data_f):
    input_size = len(data_f.readline().split(';'))
    data_f.seek(0)

    num_samples = sum([1 for _ in data_f])
    data_f.seek(0)

    return input_size, num_samples


In [ ]:
with open(RAW_DATA_PATH, 'r') as data_f:
    with open(PREPROCESSED_DATA_PATH, 'w+') as out_f:
        # Read headings
        columns = data_f.readline().strip('\n').split(',')
        col_values = defaultdict(list)
        last_idx = 0

        log('The following columns are filtered: %s' % ', '.join(STRIP_COLS))

        # Find all unique values for each row in the dataset
        # and store them in col_values.
        for i, line in enumerate(data_f):
            sample_values = line.strip('\n').split(',')

            for c, v in zip(columns, sample_values):
                if c in STRIP_COLS: continue
                if v not in col_values[c]: col_values[c].append(v)

            last_idx = i

        log('The number of distinct values for each column are:\n')

        sum_lines = last_idx+1
        sum_vec_entries = 0

        for c, v in col_values.items():
            log('  %s = %d' % (c, len(v)))
            sum_vec_entries += len(v)

        log('The generated vectors will have a total of %d entries each' % sum_vec_entries)
        log('The dataset has %i samples' % sum_lines)

        data_f.seek(0)
        data_f.readline() # skip headings after seek(0)

        start_time = time.time()
        curr_idx = 0
        temp_x = []

        for i, line in enumerate(data_f):
            sample_values = line.strip('\n').split(',')
            sample_vec = np.zeros(sum_vec_entries)
            idx_offset = 0

            for c, v in zip(columns, sample_values):
                if c in STRIP_COLS: continue
                sample_vec[col_values[c].index(v)+idx_offset] = 1
                idx_offset += len(col_values[c])

            temp_x.append(sample_vec)

            if (i+1) % 100000 == 0 or (i+1) == sum_lines:
                temp_x = np.array(temp_x)
                np.random.shuffle(temp_x)

                log('Processed %i samples (%.1f%%)...' % (i+1, 100*(float(i+1)/sum_lines)))
                log('Storing collected data in CSV file...')

                temp_x_str = []

                for i in range(temp_x.shape[0]):
                    temp_x_str.append(';'.join(map(str, map(int, temp_x[i]))))
           
                out_f.write('%s\n' % '\n'.join(temp_x_str))

                curr_idx += temp_x.shape[0]

                log('Stored data successfully! (Took %.2fs)' % (time.time() - start_time))
                start_time = time.time()
                temp_x = []

        log('Successfully stored preprocessed samples in: %s' % PREPROCESSED_DATA_PATH)


[INFO][2017-05-26 16:42:04] The following columns are filtered: Record ID, Agency Name, Agency Code, Year, Month, Record Source
[INFO][2017-05-26 16:42:15] The number of distinct values for each column are:

[INFO][2017-05-26 16:42:15]   Agency Type = 15
[INFO][2017-05-26 16:42:15]   City = 1784
[INFO][2017-05-26 16:42:15]   State = 59
[INFO][2017-05-26 16:42:15]   Incident = 1012
[INFO][2017-05-26 16:42:15]   Crime Type = 20
[INFO][2017-05-26 16:42:15]   Crime Solved = 4
[INFO][2017-05-26 16:42:15]   Victim Sex = 5
[INFO][2017-05-26 16:42:15]   Victim Age = 104
[INFO][2017-05-26 16:42:15]   Victim Race = 58
[INFO][2017-05-26 16:42:15]   Victim Ethnicity = 7
[INFO][2017-05-26 16:42:15]   Perpetrator Sex = 4
[INFO][2017-05-26 16:42:15]   Perpetrator Age = 104
[INFO][2017-05-26 16:42:15]   Perpetrator Race = 42
[INFO][2017-05-26 16:42:15]   Perpetrator Ethnicity = 6
[INFO][2017-05-26 16:42:15]   Relationship = 29
[INFO][2017-05-26 16:42:15]   Weapon = 27
[INFO][2017-05-26 16:42:15]   Vic

Das Skript konvertiert alle Morde in _database.csv_ in One-Hot Encoded Vektoren und speichert diese in einer CSV Datei. Diese können dann verwendet werden um damit den _AutEncoder_ zu trainieren und danach Embeddings für jeden einzelnen Mord zu generieren.

Beim Preprocessen werden alle Spalten berücksichtigt ausser diejenigen, welche in der Liste STRIP_COLS explizit ausgeschlossen werden. Das ganz funktioniert so, dass zuerst für jede Spalte eruiert wird, wieviele unterschiedliche Werte es pro Spalte hat. Wenn für eine Spalte _n_ verschiedene Werte vorhanden sind, so werden für die Darstellung im One-Hot Vektor entsprechend _n_ Werte für diese Spalte benötigt. Die resultierenden Vektoren sind also 

$$\sum_{c \in Columns} \operatorname{classes}(c)$$

lang, wobei _C_ für die Menge aller Spalten und _classes_ für die Anzahl unterschiedlicher Werte für die Spalte _c_ steht. Pro Zeile und Spalte werden dann diejenigen Werte, welche in der jeweiligen Zeile stehen auf _1_ gesetzt, alle anderen werden auf _0_ belassen. Als nächste folgt der Code in [TensorFlow](https://www.tensorflow.org/), welcher für die Implementation des _AutoEncoder_ und des _VariationalAutoEncoder_ zuständig ist.

Prinzipell sind _AutoEncoder_ eine spezielle Art von Neuronalen Netzen (NN), welche dafür zuständig sind, eine effiziente Codierung der Eingabedaten zu lernen. Der Aufbau ist so, dass die Eingabedaten als One-Hote Encoded Vektoren (im Bild unten _x_) über den Input Layer in das NN eingespeist wird. Diese werden dann mithilfe einer Multiplikation mit einer Gewichts-Matrix in den Hidden Layer projeziert. Dieser ist im Falle von AutoEncodern **immer** kleiner wie der Input Layer, weil das NN ja eine effiziente Codierung der Eingabedaten lernen soll. Am Ende wird das NN mithilfe von Gradient-Descent mit dem _Adam_ Optimierer darauf trainiert, aus der codierten Darstellung der Eingabedaten (im Bild unten _z_) wieder die Eingabedaten _x_ zu rekonstruieren. Die generierten Darstellungen _z_ können dann als Embeddings der Eingabedaten in einem _m_-dimensionalen Vektorraum aufgefasst werden, wobei _m_ der grösse des Hidden Layer in der Mitte entspricht. Auf diese Embeddings können wir dann später Clustering-Algorithmen anwenden um festzustellen, welche Verbrechen im eingebetteten Vektor-Raum nahe beieinander liegen.

![AutoEncoder Struktur](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png)

Der Unterschied eines "normalen" _AutoEncoder_ zu einem _VariationalAutoEncoder_ liegt darin, dass ...
Für eine gute Einführung in _VariationalAutoEncoder_ kann [dieses](http://kvfrans.com/variational-autoencoders-explained/) Tutorial hinzugezogen werden.

Wir werden alle Experimente mit beiden Varianten _AutoEncoder_ und _VariationalAutoEncoder_ durchführen. Unten folgt die Defintion der Modelle in Python mithilfe von TensorFlow.

In [ ]:
import tensorflow as tf

# The current version of the autoencoder to use
CurrentAutoEncoder = AutoEncoder

class AutoEncoder(object):
    def __init__(self, input_size, hidden_size, session):
        '''Initializes a new instance of the VariationalAutoencoder class.'''
        self.session = session
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights = {}
        self.transfer_fn = tf.nn.softplus

        self.__initialize()
        self.__build()

    def __initialize(self):
        '''Initializes the weights needed to build the computational graph.'''
        weights = {}

        weights['weights_1'] = tf.get_variable('weights_1',[self.input_size, self.hidden_size])
        weights['bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['weights_2'] = tf.Variable(tf.zeros([self.hidden_size, self.input_size], dtype=tf.float32))
        weights['bias_2'] = tf.Variable(tf.zeros([self.input_size], dtype=tf.float32))

        self.weights = weights

    def get_optimizer(self):
        '''Returns the optimizer for this instance.'''
        return self.optimizer

    def get_loss(self):
        '''Returns the loss function for this instance.'''
        return self.loss_fn

    def get_weights_and_biases(self):
        '''Returns the weights and biases of this instance.'''
        return self.weights

    def get_internal_representation(self):
        '''Returns the internal, embedded representation variables.'''
        return self.hidden

    def batch_fit(self, input):
        '''Fits the model to the given batch.'''
        loss, _ = self.session.run((self.loss_fn, self.optimizer),
                                   feed_dict={self.input: input})
        return loss

    def transform(self, input):
        return self.session.run(self.hidden, feed_dict={self.input: input})

    def __build(self):
        '''Builds the computational graph.'''
        self.input = tf.placeholder(tf.float32, [None, self.input_size])

        hidden_1_result = tf.matmul(self.input, self.weights['weights_1'])
        self.hidden = self.transfer_fn(tf.add(hidden_1_result,
                                              self.weights['bias_1']))

        reconstruction_result = tf.matmul(self.hidden, self.weights['weights_2'])
        self.reconstruction = tf.add(reconstruction_result, self.weights['bias_2'])

        diff = tf.subtract(self.reconstruction, self.input)
        self.loss_fn = 0.5 * tf.reduce_sum(tf.pow(diff, 2.0))
        self.optimizer_fn = tf.train.AdamOptimizer(learning_rate=0.001)
        self.optimizer = self.optimizer_fn.minimize(self.loss_fn)

class VariationalAutoencoder(object):
    def __init__(self, input_size, hidden_size, session):
        '''Initializes a new instance of the VariationalAutoencoder class.'''
        self.session = session
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights = {}

        self.__initialize()
        self.__build()

    def __initialize(self):
        '''Initializes the weights needed to build the computational graph.'''
        weights = {}

        weights['weights_1'] = tf.get_variable('weights_1',[self.input_size, self.hidden_size])
        weights['log_sigma_weights_1'] = tf.get_variable('log_sigma_weights_1', [self.input_size, self.hidden_size])
        weights['bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['log_sigma_bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['weights_2'] = tf.Variable(tf.zeros([self.hidden_size, self.input_size], dtype=tf.float32))
        weights['bias_2'] = tf.Variable(tf.zeros([self.input_size], dtype=tf.float32))

        self.weights = weights

    def get_optimizer(self):
        '''Returns the optimizer for this instance.'''
        return self.optimizer

    def get_loss(self):
        '''Returns the loss function for this instance.'''
        return self.loss_fn

    def get_weights(self):
        '''Returns the weights of this instance.'''
        return self.weights

    def get_internal_representation(self):
        '''Returns the internal, embedded representation variables.'''
        return self.z

    def batch_fit(self, input):
        '''Fits the model to the given batch.'''
        loss, _ = self.session.run((self.loss_fn, self.optimizer),
                                   feed_dict={self.input: input})
        return loss

    def transform(self, input):
        return self.session.run(self.z_mean, feed_dict={self.input: input})

    def partial_fit(self, X):
        loss, opt = self.sess.run((self.loss_fn, self.optimizer), feed_dict={self.x: X})
        return loss

    def transform(self, X):
        return self.sess.run(self.z_mean, feed_dict={self.x: X})

    def __build(self):
        '''Builds the computational graph.'''
        self.input = tf.placeholder(tf.float32, [None, self.input_size])

        hidden_1_result = tf.matmul(self.input, self.weights['weights_1'])
        self.z_mean = tf.add(hidden_1_result, self.weights['log_sigma_bias_1'])

        log_sigma_result = tf.matmul(self.input, self.weights['log_sigma_weights_1'])
        self.z_log_sigma_sq = tf.add(log_sigma_result, self.weights['log_sigma_bias_1'])

        eps = tf.random_normal(tf.stack([tf.shape(self.input)[0], self.hidden_size]), 0, 1, dtype=tf.float32)
        self.z = tf.add(self.z_mean, tf.multiply(tf.sqrt(tf.exp(self.z_log_sigma_sq)), eps))

        y_result = tf.matmul(self.z, self.weights['weights_2'])
        self.y = tf.add(y_result, self.weights['bias_2'])

        reconstruction_loss = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.y, self.input), 2.0))
        latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq \
                                           - tf.square(self.z_mean) \
                                           - tf.exp(self.z_log_sigma_sq), 1)

        self.loss_fn = tf.reduce_mean(reconstruction_loss + latent_loss)
        self.optimizer_fn = tf.train.AdamOptimizer(learning_rate=0.001)
        self.optimizer = self.optimizer_fn.minimize(self.loss_fn)


Nach der Definition der Modelle folgt der Code um diese zu trainieren. Das Training durchzuführen dauert entsprechend lange, auf einer mittelmässigen GPU benötigt es ca. einen Tag Rechenzeit. Im folgenden Abschnitt ist der Code, welcher zuständig ist für das Training des _AutoEncoder_, ersichtlich:

In [ ]:
encoder_type = camel_to_sneak(CurrentAutoEncoder.__name__)
time_stamp = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())
result_name = '%s-%s-results/' % (time_stamp, encoder_type)
result_path = path.join(RESULTS_DIR, result_name)
loss_track = []

with open(PREPROCESSED_DATA_PATH, 'r') as train_f:
    input_size, num_samples = get_input_size_and_length(train_f)

    log('Starting training with a %s' % CurrentAutoEncoder.__name__)

    autoencoder = CurrentAutoEncoder(input_size, HIDDEN_SIZE, session=session)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=3)

    session.run(tf.global_variables_initializer())

    for epoch in range(TRAINING_EPOCHS):
        log('Starting epoch #%d' % (epoch+1))
        num_batches = int(num_samples / BATCH_SIZE)
        avg_loss = 0

        for num_batch in range(num_batches):
            batch_x = get_next_batch(train_f, BATCH_SIZE)
            loss = autoencoder.batch_fit(batch_x)
            avg_loss += (loss / num_samples) * BATCH_SIZE

            if (num_batch+1) % DISPLAY_BATCH == 0 or (num_batches-num_batch) < 5:
                log('Batch #%d of #%d, loss = %.5f' % (num_batch+1, num_batches, loss))

        if (epoch+1) % DISPLAY_EPOCH == 0 or (epoch+1) == TRAINING_EPOCHS:
            log('Epoch #%d of #%d, loss = %.5f' % (epoch+1, TRAINING_EPOCHS, avg_loss))
            saver.save(session, result_path)


Nachdem der _AutoEncoder_ trainiert wurde kann dieser verwendet werden um damit die Embeddings für die einzelnen Verbrechen zu generieren:

In [ ]:
with open(PREPROCESSED_DATA_PATH, 'r') as samples_f:
    with open(emb_out_path, 'w+') as emb_f:
        input_size, num_samples = get_input_size_and_length(train_f)
        log('Restoring model from %s' % model_path)

        autoencoder = CurrentAutoEncoder(input_size, HIDDEN_SIZE, session=session)

        saver = tf.train.Saver(tf.global_variables(), max_to_keep=3)
        saver.restore(session, model_path)

        log('Finished restoring the model')
        log('Starting to embed the samples in %s' % samples_path)

        num_batches = int(num_samples / BATCH_SIZE)

        for num_batch in range(num_batches):
            batch_x = get_next_batch(samples_data, num_batch)
            batch_y = autoencoder.transform(batch_x)

            for y in batch_y:
                emb_f.write('%s\n' % ';'.join(map(str, y)))

            if (num_batch+1) % DISPLAY_BATCH == 0:
                log('Processed %d of %d samples' % (num_batch+1, num_batches))


Nun, da wir die Embeddings für alle Verbrechen generiert haben, können wir damit starten das K-Means Clustering auf diese anzuwenden, als auch auf die ursprünglichen One-Hot Encoded Vektoren, welche verwendet wurden um den _AutoEncoder_ zu trainieren: